## <div id='kakkuro' />カックロ

In [69]:
#!pip3 install pulp ortoolpy
import re, numpy as np, pandas as pd
from pulp import LpProblem, lpDot, lpSum, value
from ortoolpy import addvar, addvars, addbinvar, addbinvars

In [70]:
# data = """\
# ###...
# ..#...
# ....##
# ##....
# ...#..
# ...###""".splitlines()

# hint_v = [ # 開始行、開始列、個数、合計
#     (0,3,4,10),
#     (0,4,2,4),
#     (0,5,2,3),
   
#     (1,0,2,3),
#     (1,1,2,4),
    
#     (2,2,4,20),
    
#     (3,4,2,17),
#     (3,5,2,16),
   
    
#     (4,0,2,17),
#     (4,1,2,16),
    
# ]
# hint_h = [ # 開始行、開始列、個数、合計
#     (0,3,3,7),
    
#     (1,0,2,4),
#     (1,3,3,6),
    
#     (2,0,4,10),
    
#     (3,2,4,19),
    
#     (4,0,3,23),
#     (4,4,2,17),
    
#     (5,0,3,24),
# ]

In [71]:
# data = """\
# #..##
# .....
# ..#..
# .....
# ##..#""".splitlines()
# hint_v = [ # 開始行、開始列、個数、合計
#     (0,1,4,10),
#     (0,2,2,4),
   
#     (1,0,3,7),
#     (1,3,4,30),
#     (1,4,3,8),
    
#     (3,2,2,9),
# ]
# hint_h = [ # 開始行、開始列、個数、合計
#     (0,1,2,3),
    
#     (1,0,5,16),   
    
#     (2,0,2,3),
#     (2,3,2,13),
    
#     (3,0,5,17),
    
#     (4,2,2,16),
   
# ]


In [72]:
data = """\
#...#....
....#....
..#...#..
.....#...
##.....##
...#.....
..#...#..
....#....
....#...#""".splitlines()

hint_v = [ # 開始行、開始列、個数、合計
    (0,1,4,27),
    (0,2,2,15),
    (0,3,5,17),
    (0,5,3,21),
    (0,6,2,4),
    (0,7,4,10),
    (0,8,4,17),
    
    (1,0,3,10),
   
    (2,4,5,34),
    
    (3,2,3,7),
    (3,6,3,21),
      
    (4,5,5,16),
    
    (5,0,4,14),
    (5,1,4,29),
    (5,7,4,30),
    (5,8,3,7),
    
    (6,3,3,20),
    
    (7,2,2,4),
    (7,6,2,5),
]
hint_h = [ # 開始行、開始列、個数、合計
    (0,1,3,19),
    (0,5,4,14),
    
    (1,0,4,12),
    (1,5,4,12),
    
    (2,0,2,11),
    (2,3,3,23),
    (2,7,2,13),
    
    (3,0,5,33),
    (3,6,3,8),
    
    (4,2,5,18),
   
    (5,0,3,13),
    (5,4,5,29),
    
    (6,0,2,16),
    (6,3,3,22),
    (6,7,2,13),
    
    (7,0,4,19),
    (7,5,4,12),
    
    (8,0,4,11),
    (8,5,3,10),
]

In [73]:
ni,nj = len(data),len(data[0])
a = pd.DataFrame([(i,j,k) for i in range(ni) for j in range(nj)
    if data[i][j]=='.' for k in range(1,10)], columns=list('行列数'))
a['Var'] = addbinvars(len(a))

In [74]:
print('ni: ',ni)
print('nj: ',nj)

ni:  9
nj:  9


In [75]:
print(a)

     行  列  数      Var
0    0  1  1  v002125
1    0  1  2  v002126
2    0  1  3  v002127
3    0  1  4  v002128
4    0  1  5  v002129
..  .. .. ..      ...
580  8  7  5  v002705
581  8  7  6  v002706
582  8  7  7  v002707
583  8  7  8  v002708
584  8  7  9  v002709

[585 rows x 4 columns]


In [76]:
m = LpProblem()
for _,v in a.groupby(['行','列']):
    m += lpSum(v.Var) == 1 
for (di,dj),hint in zip([(1,0),(0,1)],[hint_v,hint_h]):
    for si,sj,nl,sm in hint:
        b = a.query(f'{si}<=行<={si+nl*di}&{sj}<=列<={sj+nl*dj}')[['数','Var']]
        for _,v in b.groupby('数'):
            m += lpSum(v.Var) <= 1 
        m += lpDot(*b.T.values) == sm 
%time m.solve()

CPU times: user 12.6 ms, sys: 3.84 ms, total: 16.4 ms
Wall time: 715 ms


1

In [77]:
a['Val'] = a.Var.apply(value)
r = a[a.Val>0.5].数[::-1].tolist()
print(re.sub('\\.', lambda _: str(r.pop()), '\n'.join(data)))

#793#8312
1362#4125
29#689#49
78459#431
##21438##
481#64982
79#976#94
1738#2361
2513#127#
